## Estatísticas

In [2]:
# %%
# --- CÉLULA 5: Código (Adaptado para novos caminhos e com filtro de 'water') ---

# Importações necessárias
import os
import geopandas as gpd
# Para garantir que o código funcione em caso de fallback de CRS
from pyproj import CRS
# O uso de 'rasterio' foi removido/simplificado, pois o caminho do raster não foi fornecido.

# --- 1. Definição dos Novos Caminhos ---
# Caminho do Vetor de Inundação (GeoJSON) - Arquivo base
inundacao_vetor_path = r'bff_prop_col_direct\inundacao_adsolo_buffer.geojson'

# Caminho do GeoPackage de Polígonos OSM - Arquivo para quantificar
osm_polygons_path = r'resultados_hidrologia_basica\osm_polygons.gpkg'

# Definição inicial (se falhar, o código tentará usar estes)
flood_analysis_gdf = gpd.GeoDataFrame(geometry=[], crs=CRS.from_epsg(4326))
osm_polygons_gdf = gpd.GeoDataFrame(geometry=[], crs=CRS.from_epsg(4326))
affected_polygons_gdf = gpd.GeoDataFrame(geometry=[], crs=CRS.from_epsg(4326))
count_affected_polygons = 0
total_polygons_aoi = 0

print("\n--- Iniciando Célula 5: Análise de Interseção (Inundação vs Polígonos OSM) e Estatísticas ---")

try:
    # --- Carregar Vetor de Inundação ---
    if not os.path.exists(inundacao_vetor_path):
        print(f"Erro: Arquivo vetorial de inundação não encontrado em '{inundacao_vetor_path}'.")
        # Define um GDF vazio com CRS de fallback
        flood_analysis_gdf = gpd.GeoDataFrame(geometry=[], crs=CRS.from_epsg(4326))
    else:
        print(f"Carregando vetor de inundação '{inundacao_vetor_path}'...")
        flood_analysis_gdf = gpd.read_file(inundacao_vetor_path)
        if len(flood_analysis_gdf) > 1:
             print("Aviso: Vetor de inundação contém múltiplas features. Dissolvendo novamente...")
             flood_analysis_gdf['temp_id'] = 1
             flood_analysis_gdf = flood_analysis_gdf.dissolve(by='temp_id')
        print(f"Vetor de inundação carregado (CRS: {flood_analysis_gdf.crs}).")


    # --- Carregar Polígonos OSM e Harmonizar CRS ---
    osm_polygons_input_path = osm_polygons_path
    if not os.path.exists(osm_polygons_input_path):
        print(f"Arquivo de polígonos OSM não encontrado em '{osm_polygons_input_path}'. Análise de interseção não será realizada.")
        # Se não há arquivo de inundação (empty), define CRS alvo para 4326.
        target_crs_fallback = flood_analysis_gdf.crs if not flood_analysis_gdf.empty and flood_analysis_gdf.crs else CRS.from_epsg(4326)
        osm_polygons_gdf_input = gpd.GeoDataFrame(geometry=[], crs=target_crs_fallback)
    else:
        osm_polygons_gdf_input = gpd.read_file(osm_polygons_input_path)
        print(f"Vetor de polígonos OSM '{osm_polygons_input_path}' carregado (CRS: {osm_polygons_gdf_input.crs}).")

    # Determinar o CRS alvo (Prioriza o CRS da inundação, se existir)
    target_crs = flood_analysis_gdf.crs if not flood_analysis_gdf.empty and flood_analysis_gdf.crs else osm_polygons_gdf_input.crs

    if osm_polygons_gdf_input.empty:
        print("GeoDataFrame de polígonos OSM vazio.")
        osm_polygons_gdf = gpd.GeoDataFrame(geometry=[], crs=target_crs)
    elif osm_polygons_gdf_input.crs != target_crs and target_crs is not None:
        print(f"Reprojetando polígonos OSM de {osm_polygons_gdf_input.crs} para {target_crs}...")
        osm_polygons_gdf = osm_polygons_gdf_input.to_crs(target_crs)
    elif target_crs is None:
         print("Aviso: Não foi possível determinar um CRS alvo para harmonização.")
         osm_polygons_gdf = osm_polygons_gdf_input
    else:
        print("CRS dos polígonos OSM e inundação são compatíveis.")
        osm_polygons_gdf = osm_polygons_gdf_input


    # <<< FILTRAGEM: Filtrar polígonos de 'water' >>>
    print(f"Total de polígonos OSM antes da filtragem: {len(osm_polygons_gdf)}")

    # Condição 1: Excluir se a coluna 'natural' existir e for 'water'
    if 'natural' in osm_polygons_gdf.columns:
        print("Filtrando polígonos com 'natural' == 'water'...")
        # Garante que a comparação seja feita com strings e em minúsculas
        osm_polygons_gdf = osm_polygons_gdf[
            osm_polygons_gdf['natural'].fillna('').astype(str).str.lower() != 'water'
        ]

    # Condição 2: Excluir se a coluna 'water' existir e tiver qualquer valor (não nulo)
    if 'water' in osm_polygons_gdf.columns:
        print("Filtrando polígonos com a tag 'water' preenchida...")
        # Mantém apenas as linhas onde a coluna 'water' é Nula (NaN)
        osm_polygons_gdf = osm_polygons_gdf[
            osm_polygons_gdf['water'].isna()
        ]

    print(f"Total de polígonos OSM após a filtragem (sem 'water'): {len(osm_polygons_gdf)}")


    # --- Análise de Interseção ---
    print("Realizando interseção espacial (Inundação vs Polígonos OSM)...")
    if not osm_polygons_gdf.empty and not flood_analysis_gdf.empty:

        # Ajuste de CRS de última hora (se um dos GDFs estava sem CRS)
        if osm_polygons_gdf.crs is None and flood_analysis_gdf.crs is not None:
            print("Reprojetando polígonos OSM para o CRS da inundação.")
            osm_polygons_gdf = osm_polygons_gdf.set_crs(flood_analysis_gdf.crs)
        elif flood_analysis_gdf.crs is None and osm_polygons_gdf.crs is not None:
            print("Reprojetando inundação para o CRS dos polígonos OSM.")
            flood_analysis_gdf = flood_analysis_gdf.set_crs(osm_polygons_gdf.crs)

        if osm_polygons_gdf.crs != flood_analysis_gdf.crs:
             print(f"Aviso: CRS não coincidem ({osm_polygons_gdf.crs} vs {flood_analysis_gdf.crs}). Reprojetando polígonos OSM para o CRS da inundação.")
             osm_polygons_gdf = osm_polygons_gdf.to_crs(flood_analysis_gdf.crs)


        # Nota: O sjoin usa o osm_polygons_gdf JÁ FILTRADO (sem 'water')
        affected_sjoin = gpd.sjoin(osm_polygons_gdf, flood_analysis_gdf, how='inner', predicate='intersects')
        # Remove duplicatas (um polígono OSM pode intersectar múltiplas partes do vetor de inundação)
        affected_polygons_gdf = affected_sjoin[~affected_sjoin.index.duplicated(keep='first')]
    else:
        print("Não há dados de polígonos OSM (pós-filtro) ou inundação para realizar a interseção.")
        # Garante que o GDF de saída tenha um CRS, se possível
        affected_polygons_gdf = gpd.GeoDataFrame(geometry=[], crs=target_crs if target_crs is not None else CRS.from_epsg(4326))


    # --- Estatísticas ---
    count_affected_polygons = len(affected_polygons_gdf)
    total_polygons_aoi = len(osm_polygons_gdf)

    # --- Imprimir Estatísticas ---
    print("-" * 30)
    print("Estatísticas da Análise de Impacto (Polígonos OSM):")
    print(f"Número total de polígonos OSM na AOI (excluindo 'water'): {total_polygons_aoi}")
    print(f"Número de polígonos OSM afetados pela inundação: {count_affected_polygons}")
    if total_polygons_aoi > 0:
        percent_affected = (count_affected_polygons / total_polygons_aoi) * 100
        print(f"Percentual de polígonos OSM afetados: {percent_affected:.2f}%")
    else:
         print(f"Percentual de polígonos OSM afetados: N/A (nenhum polígono OSM na AOI)")
    print("-" * 30)

    print("Célula 5 concluída com sucesso!")
    print("Variáveis `flood_analysis_gdf`, `osm_polygons_gdf`, `affected_polygons_gdf`, `count_affected_polygons`, `total_polygons_aoi` disponíveis.")

except Exception as e:
    print(f"Erro inesperado na Célula 5: {e}")
    # Define variáveis como vazias/zero em caso de erro
    flood_analysis_gdf = gpd.GeoDataFrame(geometry=[])
    osm_polygons_gdf = gpd.GeoDataFrame(geometry=[])
    affected_polygons_gdf = gpd.GeoDataFrame(geometry=[])
    count_affected_polygons = 0
    total_polygons_aoi = 0



--- Iniciando Célula 5: Análise de Interseção (Inundação vs Polígonos OSM) e Estatísticas ---
Carregando vetor de inundação 'bff_prop_col_direct\inundacao_adsolo_buffer.geojson'...


C:\Users\G15-PHCCG\miniconda3\envs\hydro_mde\lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Several features with id = 1 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


Aviso: Vetor de inundação contém múltiplas features. Dissolvendo novamente...
Vetor de inundação carregado (CRS: EPSG:4326).
Vetor de polígonos OSM 'resultados_hidrologia_basica\osm_polygons.gpkg' carregado (CRS: EPSG:4326).
CRS dos polígonos OSM e inundação são compatíveis.
Total de polígonos OSM antes da filtragem: 494
Filtrando polígonos com 'natural' == 'water'...
Filtrando polígonos com a tag 'water' preenchida...
Total de polígonos OSM após a filtragem (sem 'water'): 479
Realizando interseção espacial (Inundação vs Polígonos OSM)...
------------------------------
Estatísticas da Análise de Impacto (Polígonos OSM):
Número total de polígonos OSM na AOI (excluindo 'water'): 479
Número de polígonos OSM afetados pela inundação: 125
Percentual de polígonos OSM afetados: 26.10%
------------------------------
Célula 5 concluída com sucesso!
Variáveis `flood_analysis_gdf`, `osm_polygons_gdf`, `affected_polygons_gdf`, `count_affected_polygons`, `total_polygons_aoi` disponíveis.


## gravar no gpkg final

In [ ]:
# %%
# --- CÉLULA 5: Código (Adaptado para novos caminhos, com filtro de 'water' E SALVAMENTO DE ESTATÍSTICAS) ---

# Importações necessárias
import os
import geopandas as gpd
# Para garantir que o código funcione em caso de fallback de CRS
from pyproj import CRS

# --- 1. Definição dos Caminhos ---
# Caminho do Vetor de Inundação (GeoJSON) - Arquivo base
inundacao_vetor_path = r'bff_prop_col_direct\inundacao_adsolo_buffer.geojson'

# Caminho do GeoPackage de Polígonos OSM - Arquivo para quantificar
osm_polygons_path = r'resultados_hidrologia_basica\osm_polygons.gpkg'

# NOVO CAMINHO: Onde o vetor de inundação com as estatísticas será salvo
output_inundacao_stats_path = r'bff_prop_col_direct\inundacao_adsolo_buffer_com_stats.geojson'

# Definição inicial (se falhar, o código tentará usar estes)
flood_analysis_gdf = gpd.GeoDataFrame(geometry=[], crs=CRS.from_epsg(4326))
osm_polygons_gdf = gpd.GeoDataFrame(geometry=[], crs=CRS.from_epsg(4326))
affected_polygons_gdf = gpd.GeoDataFrame(geometry=[], crs=CRS.from_epsg(4326))
count_affected_polygons = 0
total_polygons_aoi = 0
saved_percent_affected = 0.0

print("\n--- Iniciando Célula 5: Análise de Interseção (Inundação vs Polígonos OSM) e Estatísticas ---")

try:
    # --- Carregar Vetor de Inundação ---
    if not os.path.exists(inundacao_vetor_path):
        print(f"Erro: Arquivo vetorial de inundação não encontrado em '{inundacao_vetor_path}'.")
        # Define um GDF vazio com CRS de fallback
        flood_analysis_gdf = gpd.GeoDataFrame(geometry=[], crs=CRS.from_epsg(4326))
    else:
        print(f"Carregando vetor de inundação '{inundacao_vetor_path}'...")
        flood_analysis_gdf = gpd.read_file(inundacao_vetor_path)
        if len(flood_analysis_gdf) > 1:
             print("Aviso: Vetor de inundação contém múltiplas features. Dissolvendo novamente...")
             flood_analysis_gdf['temp_id'] = 1
             flood_analysis_gdf = flood_analysis_gdf.dissolve(by='temp_id')
        print(f"Vetor de inundação carregado (CRS: {flood_analysis_gdf.crs}).")


    # --- Carregar Polígonos OSM e Harmonizar CRS ---
    osm_polygons_input_path = osm_polygons_path
    if not os.path.exists(osm_polygons_input_path):
        print(f"Arquivo de polígonos OSM não encontrado em '{osm_polygons_input_path}'. Análise de interseção não será realizada.")
        # Se não há arquivo de inundação (empty), define CRS alvo para 4326.
        target_crs_fallback = flood_analysis_gdf.crs if not flood_analysis_gdf.empty and flood_analysis_gdf.crs else CRS.from_epsg(4326)
        osm_polygons_gdf_input = gpd.GeoDataFrame(geometry=[], crs=target_crs_fallback)
    else:
        osm_polygons_gdf_input = gpd.read_file(osm_polygons_input_path)
        print(f"Vetor de polígonos OSM '{osm_polygons_input_path}' carregado (CRS: {osm_polygons_gdf_input.crs}).")

    # Determinar o CRS alvo (Prioriza o CRS da inundação, se existir)
    target_crs = flood_analysis_gdf.crs if not flood_analysis_gdf.empty and flood_analysis_gdf.crs else osm_polygons_gdf_input.crs

    if osm_polygons_gdf_input.empty:
        print("GeoDataFrame de polígonos OSM vazio.")
        osm_polygons_gdf = gpd.GeoDataFrame(geometry=[], crs=target_crs)
    elif osm_polygons_gdf_input.crs != target_crs and target_crs is not None:
        print(f"Reprojetando polígonos OSM de {osm_polygons_gdf_input.crs} para {target_crs}...")
        osm_polygons_gdf = osm_polygons_gdf_input.to_crs(target_crs)
    elif target_crs is None:
         print("Aviso: Não foi possível determinar um CRS alvo para harmonização.")
         osm_polygons_gdf = osm_polygons_gdf_input
    else:
        print("CRS dos polígonos OSM e inundação são compatíveis.")
        osm_polygons_gdf = osm_polygons_gdf_input


    # <<< FILTRAGEM: Filtrar polígonos de 'water' >>>
    print(f"Total de polígonos OSM antes da filtragem: {len(osm_polygons_gdf)}")

    # Condição 1: Excluir se a coluna 'natural' existir e for 'water'
    if 'natural' in osm_polygons_gdf.columns:
        print("Filtrando polígonos com 'natural' == 'water'...")
        # Garante que a comparação seja feita com strings e em minúsculas
        osm_polygons_gdf = osm_polygons_gdf[
            osm_polygons_gdf['natural'].fillna('').astype(str).str.lower() != 'water'
        ]

    # Condição 2: Excluir se a coluna 'water' existir e tiver qualquer valor (não nulo)
    if 'water' in osm_polygons_gdf.columns:
        print("Filtrando polígonos com a tag 'water' preenchida...")
        # Mantém apenas as linhas onde a coluna 'water' é Nula (NaN)
        osm_polygons_gdf = osm_polygons_gdf[
            osm_polygons_gdf['water'].isna()
        ]

    print(f"Total de polígonos OSM após a filtragem (sem 'water'): {len(osm_polygons_gdf)}")


    # --- Análise de Interseção ---
    print("Realizando interseção espacial (Inundação vs Polígonos OSM)...")
    if not osm_polygons_gdf.empty and not flood_analysis_gdf.empty:

        # Ajuste de CRS de última hora (se um dos GDFs estava sem CRS)
        if osm_polygons_gdf.crs is None and flood_analysis_gdf.crs is not None:
            print("Reprojetando polígonos OSM para o CRS da inundação.")
            osm_polygons_gdf = osm_polygons_gdf.set_crs(flood_analysis_gdf.crs)
        elif flood_analysis_gdf.crs is None and osm_polygons_gdf.crs is not None:
            print("Reprojetando inundação para o CRS dos polígonos OSM.")
            flood_analysis_gdf = flood_analysis_gdf.set_crs(osm_polygons_gdf.crs)

        if osm_polygons_gdf.crs != flood_analysis_gdf.crs:
             print(f"Aviso: CRS não coincidem ({osm_polygons_gdf.crs} vs {flood_analysis_gdf.crs}). Reprojetando polígonos OSM para o CRS da inundação.")
             osm_polygons_gdf = osm_polygons_gdf.to_crs(flood_analysis_gdf.crs)


        # Nota: O sjoin usa o osm_polygons_gdf JÁ FILTRADO (sem 'water')
        affected_sjoin = gpd.sjoin(osm_polygons_gdf, flood_analysis_gdf, how='inner', predicate='intersects')
        # Remove duplicatas (um polígono OSM pode intersectar múltiplas partes do vetor de inundação)
        affected_polygons_gdf = affected_sjoin[~affected_sjoin.index.duplicated(keep='first')]
    else:
        print("Não há dados de polígonos OSM (pós-filtro) ou inundação para realizar a interseção.")
        # Garante que o GDF de saída tenha um CRS, se possível
        affected_polygons_gdf = gpd.GeoDataFrame(geometry=[], crs=target_crs if target_crs is not None else CRS.from_epsg(4326))


    # --- Estatísticas ---
    count_affected_polygons = len(affected_polygons_gdf)
    total_polygons_aoi = len(osm_polygons_gdf)

    # Cálculo do percentual para impressão e salvamento
    if total_polygons_aoi > 0:
        saved_percent_affected = (count_affected_polygons / total_polygons_aoi) * 100
    else:
        saved_percent_affected = 0.0


    # --- Imprimir Estatísticas ---
    print("-" * 30)
    print("Estatísticas da Análise de Impacto (Polígonos OSM):")
    print(f"Número total de polígonos OSM na AOI (excluindo 'water'): {total_polygons_aoi}")
    print(f"Número de polígonos OSM afetados pela inundação: {count_affected_polygons}")

    if total_polygons_aoi > 0:
        print(f"Percentual de polígonos OSM afetados: {saved_percent_affected:.2f}%")
    else:
         print(f"Percentual de polígonos OSM afetados: N/A (nenhum polígono OSM na AOI)")
    print("-" * 30)

    # --- NOVO BLOCO: Anexar estatísticas ao vetor de inundação e salvar ---
    print(">>> Adicionando estatísticas no vetor de inundação... <<<")
    if not flood_analysis_gdf.empty and len(flood_analysis_gdf) >= 1:

        # Certifica-se de que estamos trabalhando com a primeira (e idealmente única, após dissolução) feature.
        first_index = flood_analysis_gdf.index[0]

        # Adiciona as novas colunas
        flood_analysis_gdf.loc[first_index, 'osm_count_total'] = total_polygons_aoi
        flood_analysis_gdf.loc[first_index, 'osm_count_affected'] = count_affected_polygons
        flood_analysis_gdf.loc[first_index, 'osm_perc_affected'] = round(saved_percent_affected, 2)

        print(f"Novas colunas adicionadas à feature de índice '{first_index}'.")

        # Salva o arquivo GeoJSON com as novas colunas
        print(f"Salvando o vetor de inundação com estatísticas em '{output_inundacao_stats_path}'...")
        # Usa driver='GeoJSON' explicitamente para garantir a saída correta
        flood_analysis_gdf.to_file(output_inundacao_stats_path, driver='GeoJSON')
        print("Vetor de inundação com estatísticas salvo com sucesso!")

    else:
        print("Aviso: O GeoDataFrame de inundação está vazio ou não possui features para anexar estatísticas.")


    print("Célula 5 concluída com sucesso!")
    print("Variáveis `flood_analysis_gdf`, `osm_polygons_gdf`, `affected_polygons_gdf`, `count_affected_polygons`, `total_polygons_aoi` disponíveis.")

except Exception as e:
    print(f"Erro inesperado na Célula 5: {e}")
    # Define variáveis como vazias/zero em caso de erro
    flood_analysis_gdf = gpd.GeoDataFrame(geometry=[])
    osm_polygons_gdf = gpd.GeoDataFrame(geometry=[])
    affected_polygons_gdf = gpd.GeoDataFrame(geometry=[])
    count_affected_polygons = 0
    total_polygons_aoi = 0